# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Webscraping Project 4 Lab

Week 4 | Day 4

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com (or other sites at your team's discretion). In the second part, the focus is on using listings with salary information to build a model and predict high or low salaries and what features are predictive of that result.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [5]:
## YOUR CODE HERE

r = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10')

In [7]:
r.request.headers

{'Connection': 'keep-alive', 'User-Agent': 'python-requests/2.10.0', 'Accept-Encoding': 'gzip, deflate', 'Cookie': 'CTK=1av1sdmfva461aqu; ctkgen=1; JSESSIONID=D377389F903555E2990CC9A50D737A5D.jasxA_ord-job30; INDEED_CSRF_TOKEN=7QfRvUK648uHzQlFtMClcHqCG0Ficnxj; BIGipServerjob=!Tx3EtNZc5Hs9JhA/0rl98CFw+W0yBr6MLuEFE65LjEd26S1ReIa+BHOOq7So4u4V+BETsQCtrP8tx5E=', 'Accept': '*/*'}

In [8]:
for k, v in r.request.headers.items():
    print(k + ':', v)

Connection: keep-alive
User-Agent: python-requests/2.10.0
Accept-Encoding: gzip, deflate
Accept: */*
Cookie: CTK=1av1sdmfva461aqu; ctkgen=1; JSESSIONID=D377389F903555E2990CC9A50D737A5D.jasxA_ord-job30; INDEED_CSRF_TOKEN=7QfRvUK648uHzQlFtMClcHqCG0Ficnxj; BIGipServerjob=!Tx3EtNZc5Hs9JhA/0rl98CFw+W0yBr6MLuEFE65LjEd26S1ReIa+BHOOq7So4u4V+BETsQCtrP8tx5E=


In [9]:
r.headers

{'Connection': 'Keep-Alive', 'X-Frame-Options': 'SAMEORIGIN, SAMEORIGIN', 'Vary': 'User-Agent,Accept-Encoding', 'Keep-Alive': 'timeout=30, max=29996', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'PREF=""; Domain=.indeed.com; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/, PREF="TM=1476458371622:L=New+York"; Version=1; Max-Age=536870911; Expires=Wed, 19-Oct-2033 10:08:02 GMT; Path=/, RQ="q=data+scientist+%2420%2C000&l=New+York"; Version=1; Max-Age=2592000; Expires=Sun, 13-Nov-2016 15:19:31 GMT; Path=/, UD="LA=1476458371:CV=1476458371:TS=1476458371:SG=dc11d8f1f69090a272a86aa08ff9b8cf"; Version=1; Max-Age=536870911; Expires=Wed, 19-Oct-2033 10:08:02 GMT; Path=/', 'Date': 'Fri, 14 Oct 2016 15:19:31 GMT', 'X-Content-Type-Options': 'nosniff', 'Server': 'Apache', 'Content-Type': 'text/html;charset=UTF-8'}

In [10]:
for k, v in r.headers.items():
    print(k + ':', v)

Date: Fri, 14 Oct 2016 15:19:31 GMT
Server: Apache
X-Frame-Options: SAMEORIGIN, SAMEORIGIN
Set-Cookie: PREF=""; Domain=.indeed.com; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/, PREF="TM=1476458371622:L=New+York"; Version=1; Max-Age=536870911; Expires=Wed, 19-Oct-2033 10:08:02 GMT; Path=/, RQ="q=data+scientist+%2420%2C000&l=New+York"; Version=1; Max-Age=2592000; Expires=Sun, 13-Nov-2016 15:19:31 GMT; Path=/, UD="LA=1476458371:CV=1476458371:TS=1476458371:SG=dc11d8f1f69090a272a86aa08ff9b8cf"; Version=1; Max-Age=536870911; Expires=Wed, 19-Oct-2033 10:08:02 GMT; Path=/
X-Content-Type-Options: nosniff
Vary: User-Agent,Accept-Encoding
Content-Encoding: gzip
Keep-Alive: timeout=30, max=29996
Connection: Keep-Alive
Transfer-Encoding: chunked
Content-Type: text/html;charset=UTF-8


In [11]:
r.content

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta http-equiv="content-type" content="text/html;charset=UTF-8">\n<!-- pll --><script type="text/javascript" src="/s/af4f8a6/en_US.js"></script>\n<link href="/s/b1a3616/jobsearch_all.css" rel="stylesheet" type="text/css">\n<link rel="alternate" type="application/rss+xml" title="Data Scientist $20,000 Jobs, Employment in New York, NY" href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&l=New+York">\n<link rel="alternate" media="handheld" href="/m/jobs?q=data+scientist+%2420%2C000&l=New+York">\n<script type="text/javascript">\n    \n    window[\'closureReadyCallbacks\'] = [];\n\n    function call_when_jsall_loaded(cb) {\n        if (window[\'closureReady\']) {\n            cb();\n        } else {\n            window[\'closureReadyCallbacks\'].push(cb);\n        }\n    }\n</script>\n\n<script type="text/javascript" src="/s/23b3014/jobsearch-all-compiled.js"></script>\n            <script type="text/javascript">\nvar pingUrlsForGA = [

In [12]:
from IPython.core.display import HTML
HTML(r.content.decode('utf-8'))

In [17]:
insoup = BeautifulSoup(r.content,'lxml')
insoup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>\n  <!-- pll -->\n  <script src="/s/af4f8a6/en_US.js" type="text/javascript">\n  </script>\n  <link href="/s/b1a3616/jobsearch_all.css" rel="stylesheet" type="text/css"/>\n  <link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&amp;l=New+York" rel="alternate" title="Data Scientist $20,000 Jobs, Employment in New York, NY" type="application/rss+xml"/>\n  <link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=New+York" media="handheld" rel="alternate"/>\n  <script type="text/javascript">\n   window[\'closureReadyCallbacks\'] = [];\n\n    function call_when_jsall_loaded(cb) {\n        if (window[\'closureReady\']) {\n            cb();\n        } else {\n            window[\'closureReadyCallbacks\'].push(cb);\n        }\n    }\n  </script>\n  <script src="/s/23b3014/jobsearch-all-compiled.js" type="text/javascript">\n  </script>\n  <script type="text/javascript">\n

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above
- Include any other features you may want to use later (e.g. summary, #of reviews...)

In [18]:
## YOUR CODE HERE
def extract_location_from_result(loc):
   return loc.findAll(class_='location')

i = 0
for d in extract_location_from_result(insoup):
   print(d)
   print('\n')
   i += 1

<span class="location">New York, NY 10036</span>


<span class="location">New York, NY</span>


<span class="location">New York, NY</span>


<span class="location" itemprop="address" itemscope="" itemtype="http://schema.org/Postaladdress"><span itemprop="addressLocality">New York, NY</span></span>


<span class="location" itemprop="address" itemscope="" itemtype="http://schema.org/Postaladdress"><span itemprop="addressLocality">New York, NY 10001 <span style="font-size: smaller">(Chelsea area)</span></span></span>


<span class="location" itemprop="address" itemscope="" itemtype="http://schema.org/Postaladdress"><span itemprop="addressLocality">New York, NY</span></span>


<span class="location" itemprop="address" itemscope="" itemtype="http://schema.org/Postaladdress"><span itemprop="addressLocality">New York, NY</span></span>


<span class="location" itemprop="address" itemscope="" itemtype="http://schema.org/Postaladdress"><span itemprop="addressLocality">New York, NY</span></span>


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Indeed.com only has salary information for an estimated 20% of job postings. You may want to add other cities to the list below to gather more data. 
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Atlanta']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [14]:
## YOUR CODE HERE